## Теоретическая часть


### 1. Ответьте на вопросы:  

#### **В чем принципиальное отличие гибридных рекомендательных систем от коллаборативной филтьтрации?**  

- Гибридная рекомендательная (LIGHT FM) система включает в себя не только матрицу интеракций, как в collaboratife filtering, а еще и content-based filtering с пользователями и товароми.
- Гибридные системы можно добавлять внешние фичи пользователь и товаров как content-based approach
- Используя внешение features LIGHT FM рещает проблеуму холондого старта используя content-based filtering
- Гибридная рекомендательная (LIGHT FM) система включает в себя не только матрицу интеракций, как в коллаботавной фильтрации, а еще и контент (информацию о пользователях и товарах). В гибридные системы (Light FM) можно добавлять внешние фичи пользователь и товаров

#### **Приведите 2-3 примера задач, в которых необходимо использовать гибридные системы**  

- Если постоянно появляются новые товары и новые пользователи, к примеру фондовая биржа.
- При работе магазина электроники имеется множество features, благодаря этим featuгes, при появление нового товара можно рекомендовать его клинтеу опираясь на спецификах похожего старого товара


### 2.  Прочитайте статью про поиск на hh.ru https://habr.com/ru/company/hh/blog/347276/

- Нам интересна именно рекомендательная система, раздел "Производительность системы" можно пропустить
- Какие основные отличия предложенной системы от тех подходов, которые мы разбирали на семинарах? 
- Какие проблемы могут возникнуть при выводе такой модели в продакшен?

Используется ансамблирование алгоритмов. А именно воронка фильтров на первом этапе (линейную и логит), чтобы отсечь все лишнее, а затем XGBoost для ранжирования. XGboost вступает, после того, как уже отфильтрованы вручную и программными методами значительная часть признаков. И на этапе ранжирования алгоритм приоритезирует рекламные вакансии, что помогает монетизировать бизнес компании.

При запуске подобной системы в продакшне основной проблемой было то, что проектируемая система требовала больше имеющихся ресурсов на существующих серверах. Ресурсы нужны были для того, что посчитать по резюме и по вакансии признаки, а потом — парные для их сочетания. Также, при включении поиска по пустому запросу нагрузка на поисковую систему должна была вырасти примерно в 6 раз, поэтому потребовалось кешировать признаки для резюме.

### 3. На вебинаре мы рассматривали модель LightFM (https://making.lyst.com/lightfm/docs/lightfm.html). 

В работе Data Scientist'а важную часть занимает research - исследование существующих архитектур и разбор научных статей, в которых они описываются. Вам предлагается изчуть оригинальную статью про LightFM https://arxiv.org/pdf/1507.08439.pdf и ответить на следующие вопросы:  

**1) Какой датасет используют авторы?** 
- MovieLens 10M датасет объединеный с Tag Genome сетом
- CrossValidated датасет

**2) Что используют в качестве признаков?** 
- Для MovieLens: жанр, и список меток (tags)
- Для CrossValidated: таги вопросов, мета юзеров (обо мне)

**3) С какими моделями сравнивают LightFM? Опишите их основные идеи кратко.**

Сравнивают с коллаборативной моделью матричной факторизации. Отличие от collaborative MF model в том, что LightFM расширяет её за счёт использования доп. признаков. Если на вход в LightFM подать только матрицу интеракций, то она сведётся к collaborative MF model.

- MF - модель колаборативной фильтрации - матричная модель User х Item с функцией сигмойды
- LSI-LR - логистическая регрессия. Основывается на факторизации матрицы контента.
- LSI-UP - гибридная модель, которая представляет профили пользователей, как линейную комбинацию предметов по векторам, затем применяет LSI к получившейся матрице, чтобы получить скрытые представления пользователей и элементов.

## Практическая часть


In [2]:
import lightfm

In [3]:
# import data
# utils functions like in webinar

### Модуль SRC

На вебинаре было рассказано про модуль src. Он приложен в материалах. Скачайте его, изучите структуру, импортируйте функции

В папку в которой хранится ноутбук проекта вставил папку SRС. Убедился в наличии файла ```init.py.``` Он нужен для того чтобы Python увидел модуль. 

Доюавил функции в ```utils. import os, sys module_path = os.path.abspath(os.path.join(os.pardir)) if module_path not in sys.path: sys.path.append(module_path)```

Эта структура добавляет в системный путь путь модуля если его ранее там не было, ориентируясь от родительской директории файла (полагая что папка модуля там же).

Можно все это сделоть проще, если закинуть все функции и классы в один файл ```XX.py```, тогда просто: ```from XX import```

По умолчанию как выше импортируем ссылаясь на конкретный файл из модуля, например: ```from src.utils import```

In [3]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

### Работа с признаками

У нас есть внешние данные. Что с ними не так? Чего не хватает?  

Проведите исследование внешних данных и составьте какие-нибудь содержательные выводы.  
Формально Вам нужно построить 3+ графиков (scatter plot, hist или что-то иное) и описать, что мы видим (например, товары такой-то категории болле часто покупаются в следующие дни недели или пользователи с большим достатком предпочитают такие-то товары).  
Исследуйте те закономерности, которые Вам интересно, чем менее тривиальный вывод получается, тем лучше! 

### LightFM

У этого алогритма есть множество параметров (item/user_alpha, loss, no_components).  
Проведите эксперименты аналогично дз 3 (подберите гипперпараметры каким удобно способои и постройте графики)  
На выходе необходимо получить pr@5 на валидации (последние 3 недели) > 17%  

У Вас, скорее всего, возникнет проблема со временем обучения. Почему они возникает?    

Попробуйте запустить алгоритм вообще без фичей или используйте только признаки с небольшим числом уникальных категорий. (item_features['commodity_desc'].unique() - 300 уникальных категорий - это очень много)

### *Отбор признаков* * 

Все данные категориальные, при ohe кодировании для товаров признаков становится невероятно много.      
Какие стратегии отбора признаков в классическом ML Вы знаете? Применимы ли они тут?  

Попробйте какие-нибудь стратегии. Удалось ли улучшить качество?

 \* *задание необязательно*
